Sentiment analysis of tweets regarding NBA games.

In [24]:
import pandas as pd
import datetime as datetime
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import math

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\micha\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Done importing.

In [2]:
tweets_df = pd.read_csv('Datasets\mavs_games_tweets_merged.csv')
tweets_df.head()

,Unnamed: 0,Datetime,tweet_id,username,text,verified,retweet_count,like_count,Date,Opposing Team,Win or Loss,Score,Record,Leading Scorer-Winner,Leading Scorer Points,Leading Scorer Loss,Leading Scorer Points.1
0,0,2022-02-09 23:58:50+00:00,1491562265316773888,TheMontyShow,I told Mavs Bulls likeliest destination Collin...,False,0,1,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2022-02-09 23:58:30+00:00,1491562180872974336,neilbobnuss,Your first counter offer gets laughed room haha,False,0,1,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2022-02-09 23:58:30+00:00,1491562180281577480,tac234,"Trade Hield , Nurkic , know players actually h...",False,0,1,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2022-02-09 23:58:25+00:00,1491562158374723585,VeritasetfidePF,"Ja hj , ninguém sabe quem é . So aqui na rede ...",False,0,4,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2022-02-09 23:58:12+00:00,1491562107258687490,VCPHoops,"Mavs Post Game Interview : Jason Kidd , Trey B...",False,0,0,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
tw_text = tweets_df['text']
tw_text.head()
tw_text

0         I told Mavs Bulls likeliest destination Collin...
1           Your first counter offer gets laughed room haha
2         Trade Hield , Nurkic , know players actually h...
3         Ja hj , ninguém sabe quem é . So aqui na rede ...
4         Mavs Post Game Interview : Jason Kidd , Trey B...
                                ...                        
176118    I ’ saying throw , ’ clear ’ substantive conve...
176119    U mean like Redick Meli ...... I 'm surprised ...
176120         I sure hope , want see Luka Mavs good things
176121    Jazz Suns Lakers Mavs Blazers Nuggets Warriors...
176122                da um mata leão nele e tranca armário
Name: text, Length: 176123, dtype: object

Creating a sentiment analyzer and appending sentiment to the main dataset

In [26]:
sia = SentimentIntensityAnalyzer()

score = []
for tweet in tqdm.tqdm(tw_text):
    tweet = str(tweet)
    sentiment = sia.polarity_scores(tweet)
    compoundScore = score.append(sentiment['compound'])

tweets_df['tweet_sentiment'] = score
tweets_df

100%|██████████| 176123/176123 [00:28<00:00, 6280.19it/s]


,Unnamed: 0,Datetime,tweet_id,username,text,verified,retweet_count,like_count,Date,Opposing Team,Win or Loss,Score,Record,Leading Scorer-Winner,Leading Scorer Points,Leading Scorer Loss,Leading Scorer Points.1,tweet_sentiment
0,0,2022-02-09 23:58:50+00:00,1491562265316773888,TheMontyShow,I told Mavs Bulls likeliest destination Collin...,False,0,1,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
1,1,2022-02-09 23:58:30+00:00,1491562180872974336,neilbobnuss,Your first counter offer gets laughed room haha,False,0,1,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7184
2,2,2022-02-09 23:58:30+00:00,1491562180281577480,tac234,"Trade Hield , Nurkic , know players actually h...",False,0,1,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.6705
3,3,2022-02-09 23:58:25+00:00,1491562158374723585,VeritasetfidePF,"Ja hj , ninguém sabe quem é . So aqui na rede ...",False,0,4,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
4,4,2022-02-09 23:58:12+00:00,1491562107258687490,VCPHoops,"Mavs Post Game Interview : Jason Kidd , Trey B...",False,0,0,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176118,176118,2021-10-20 00:02:19+00:00,1450613312778031108,ChrisGallegosNV,"I ’ saying throw , ’ clear ’ substantive conve...",False,0,0,2021-10-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9476
176119,176119,2021-10-20 00:01:13+00:00,1450613037568872454,MensaNinja,U mean like Redick Meli ...... I 'm surprised ...,False,0,0,2021-10-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6096
176120,176120,2021-10-20 00:01:02+00:00,1450612991649632262,ZachPrell,"I sure hope , want see Luka Mavs good things",False,0,1,2021-10-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8126
176121,176121,2021-10-20 00:00:54+00:00,1450612957193261056,Aaron_Del24,Jazz Suns Lakers Mavs Blazers Nuggets Warriors...,False,0,0,2021-10-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000


Saving the new dataset

In [28]:
#tweets_df.to_csv('Datasets\\tweets_games_sentiment.csv')

In [5]:
tweets_sent = pd.read_csv('Datasets\\tweets_games_sentiment.csv')
tweets_sent

,Unnamed: 0,Unnamed: 0.1,Datetime,tweet_id,username,text,verified,retweet_count,like_count,Date,Opposing Team,Win or Loss,Score,Record,Leading Scorer-Winner,Leading Scorer Points,Leading Scorer Loss,Leading Scorer Points.1,tweet_sentiment
0,0,0,2022-02-09 23:58:50+00:00,1491562265316773888,TheMontyShow,I told Mavs Bulls likeliest destination Collin...,False,0,1,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
1,1,1,2022-02-09 23:58:30+00:00,1491562180872974336,neilbobnuss,Your first counter offer gets laughed room haha,False,0,1,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7184
2,2,2,2022-02-09 23:58:30+00:00,1491562180281577480,tac234,"Trade Hield , Nurkic , know players actually h...",False,0,1,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.6705
3,3,3,2022-02-09 23:58:25+00:00,1491562158374723585,VeritasetfidePF,"Ja hj , ninguém sabe quem é . So aqui na rede ...",False,0,4,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
4,4,4,2022-02-09 23:58:12+00:00,1491562107258687490,VCPHoops,"Mavs Post Game Interview : Jason Kidd , Trey B...",False,0,0,2022-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176118,176118,176118,2021-10-20 00:02:19+00:00,1450613312778031108,ChrisGallegosNV,"I ’ saying throw , ’ clear ’ substantive conve...",False,0,0,2021-10-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9476
176119,176119,176119,2021-10-20 00:01:13+00:00,1450613037568872454,MensaNinja,U mean like Redick Meli ...... I 'm surprised ...,False,0,0,2021-10-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6096
176120,176120,176120,2021-10-20 00:01:02+00:00,1450612991649632262,ZachPrell,"I sure hope , want see Luka Mavs good things",False,0,1,2021-10-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8126
176121,176121,176121,2021-10-20 00:00:54+00:00,1450612957193261056,Aaron_Del24,Jazz Suns Lakers Mavs Blazers Nuggets Warriors...,False,0,0,2021-10-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000


In [50]:
#TODO: fix this thing. Fill the nan's with the result of the next game that happens.

game_wl = tweets_sent['Win or Loss']
following_result = game_wl

record = pd.read_csv('Datasets\\Mavs_game_Data.csv')
record = record['Win or Loss']
iter_record = iter(record)
nextval = next(iter_record)

final_wl = []


for i, value in following_result.items():
    try:
        if math.isnan(value):
            final_wl.append(nextval)
    except TypeError:
        final_wl.append(value)



final_wl




ValueError: Length of values (10039011) does not match length of index (176123)